![Logo BV IBMEC](https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/logo-bv-ibmec-notebooks.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ian-iania/IBMEC-BV-Modelos-Preditivos/blob/main/notebooks/NB01_Regression.ipynb)

# NB01 - Regressão e Séries Temporais (FP&A Banco BV)

**Objetivo:** aplicar regressão e séries temporais em dados sintéticos de FP&A do BV, com linguagem simples e foco em negócio.
**Vamos comparar baseline e modelos com split temporal e métricas MAE/RMSE, em fluxo linear.**
**A ideia é entender o processo sem exigir conhecimento prévio de programação.**


## 1) Setup do notebook

### O que vamos fazer
Importar as bibliotecas da aula.

### Por que isso importa para FP&A
Sem essas bibliotecas não conseguimos carregar dados, treinar modelos e visualizar resultados.

### O que você deve ver no output
Sem erro na execução.


## 1.2) Mapa da aula (roteiro de sala)

Tempo sugerido (walkthrough):
- **Bloco A (X→y linear):** 25 min
- **Bloco B (X→y não-linear):** 30 min
- **Bloco C (t→y séries temporais):** 25 min
- **Fechamento executivo:** 10 min

Objetivo prático por bloco:
- A: entender pipeline básico e baseline
- B: ver quando não-linearidade muda o ranking
- C: comparar abordagens temporais e exógenas


## 1.3) Como não se perder durante o walkthrough

- Rode as células **na ordem**.
- Se aparecer erro, volte **1 ou 2 células** e rode de novo.
- Compare sempre com o output esperado na explicação Markdown.
- Se der pane geral: `Ambiente de execução -> Reiniciar sessão` e rode novamente de cima.


## 1.4) Glossário rápido (linguagem de negócio)

- **MAE:** erro médio absoluto (quanto erramos, em média)
- **RMSE:** erro com peso maior para erros grandes
- **Resíduo:** `Real - Predito`
- **Overfitting:** modelo “decorou” treino e piorou no teste
- **Exógena:** variável externa (ex.: Selic) usada no modelo temporal
- **Regime:** faixa de comportamento diferente (ex.: Selic alta vs baixa)


In [ ]:
import pandas as pd  # biblioteca para carregar e manipular tabelas (DataFrames)
import numpy as np  # biblioteca para operações numéricas
import matplotlib.pyplot as plt  # biblioteca para criar gráficos
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet  # modelos lineares
from sklearn.tree import DecisionTreeRegressor  # modelo de árvore de decisão
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor  # modelos de ensemble baseados em árvores
from sklearn.metrics import mean_absolute_error, mean_squared_error  # métricas de avaliação MAE e RMSE
from statsmodels.tsa.holtwinters import ExponentialSmoothing  # modelo ETS (Holt-Winters)
from statsmodels.tsa.statespace.sarimax import SARIMAX  # modelo ARIMA/ARIMAX


### Visualização (Plotly)

- Plotly é interativo (hover/zoom).
- Ajuda FP&A a explorar cenários rapidamente.
- Nos gráficos a seguir, vamos usar Plotly como padrão visual.

In [ ]:
import plotly.express as px  # biblioteca de gráficos interativos de alto nível
import plotly.graph_objects as go  # biblioteca para camadas extras (ex.: linha 45 graus)
import plotly.io as pio  # controle de renderização de gráficos

pio.renderers.default = "colab"  # define renderização padrão para Google Colab
print("Plotly pronto para uso no notebook.")  # confirma setup visual

---
## 1.1) Dois caminhos de previsão em FP&A: Supervisionados (X→Y) vs Séries temporais (t→Y)

### ✅ Modelos supervisionados (X → Y)
- Temos um alvo `y` (ex.: originação mensal) e drivers `X` (Selic, LTV, mix, marketing…).
- O modelo aprende a relação **X → y** no histórico.
- **Treino vs Teste (no tempo):** treinamos no passado e avaliamos no período futuro (o que FP&A realmente precisa).

### ✅ Modelos de séries temporais (t → Y)
- O principal driver é o **tempo**: tendência, sazonalidade e “memória” do histórico.
- Algumas abordagens aceitam drivers externos (ex.: Selic) — isso aparece em **ARIMAX** (e pode aparecer no Prophet).

> Mensagem FP&A: não existe “modelo melhor sempre”. Existe o melhor modelo para o padrão do dado e para o objetivo (forecast robusto e defendível).
---


## 2) PARTE A - Regressão com dataset linear (X -> y)

### O que vamos fazer
Carregar dataset linear, montar treino/teste e comparar baseline + modelos.

### Por que isso importa para FP&A
Esse tipo de modelo ajuda a explicar e prever a originação com base nos drivers.


In [ ]:
url_linear = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_regressao_linear.csv'  # URL do dataset linear
df_lin = pd.read_csv(url_linear)  # leitura do CSV para DataFrame
df_lin['ds'] = pd.to_datetime(df_lin['ds'])  # conversão da data para datetime
df_lin.head()  # exibe as primeiras linhas da base


### 2.1 Significado das colunas

- `ds`: data de referência mensal (início do mês)
- `y`: originação mensal em R$ milhões (**variável alvo**)
- `selic`: taxa SELIC
- `desemprego`: taxa de desemprego
- `ltv_medio`: LTV médio
- `spread`: spread médio
- `marketing`: índice sintético de esforço comercial
- `mix_auto`: participação de mix auto (0 a 1)
- `mes`: mês numérico (1 a 12)

Observação: `ds` e `y` são nomes comuns em modelagem (inclusive Prophet).


### 2.2 Inspeção da base (saídas separadas)

Agora vamos checar tamanho, colunas e tipos em células separadas para não misturar output.


In [ ]:
df_lin.shape  # mostra quantidade de linhas e colunas da base


### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
list(df_lin.columns)  # lista os nomes de todas as colunas


### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
df_lin.dtypes  # mostra o tipo de dado de cada coluna


### 2.3 Visualização inicial

### O que vamos fazer
Ver `y` ao longo do tempo e `selic` versus `y`.

### O que você deve ver no output
Um gráfico de linha e um de dispersão.


In [ ]:
fig_a_line = px.line(  # cria gráfico de linha interativo da série temporal da PARTE A
    df_lin,
    x='ds',
    y='y',
    title='PARTE A — Originação (y) ao longo do tempo',
    labels={'ds': 'Data', 'y': 'y (R$ milhões)'},
    template='plotly_white'
)
fig_a_line.show()  # exibe gráfico interativo com hover e zoom

### Gráfico de dispersão inicial (PARTE A)

Agora observamos a relação entre SELIC e y com hover interativo para leitura ponto a ponto.

In [ ]:
fig_a_scatter = px.scatter(  # cria dispersão interativa entre SELIC e originação
    df_lin,
    x='selic',
    y='y',
    title='PARTE A — SELIC vs Originação (y)',
    labels={'selic': 'SELIC', 'y': 'y (R$ milhões)'},
    hover_data=['selic', 'y'],
    template='plotly_white'
)
fig_a_scatter.show()  # exibe gráfico com hover para leitura ponto a ponto

### 2.4 Split temporal e baseline

### O que vamos fazer
Separar treino/teste no tempo (80%/20%, sem embaralhar) e calcular baseline naive.

### Por que isso importa para FP&A
No mundo real, o modelo sempre prevê períodos futuros com base no passado.


In [ ]:
features = ['selic', 'desemprego', 'ltv_medio', 'spread', 'marketing', 'mix_auto', 'mes']  # lista de variáveis explicativas
split_idx = int(len(df_lin) * 0.8)  # define ponto de corte para treino e teste
train_lin = df_lin.iloc[:split_idx].copy()  # seleciona linhas de treino
train_lin = train_lin.sort_values('ds')  # garante ordenação temporal no treino
test_lin = df_lin.iloc[split_idx:].copy()  # seleciona linhas de teste
test_lin = test_lin.sort_values('ds')  # garante ordenação temporal no teste
X_train_lin = train_lin[features]  # cria matriz X de treino
X_test_lin = test_lin[features]  # cria matriz X de teste
y_train_lin = train_lin['y']  # cria vetor y de treino
y_test_lin = test_lin['y']  # cria vetor y de teste
print('Treino:', X_train_lin.shape, 'Teste:', X_test_lin.shape)  # imprime tamanhos de treino e teste
print('Período treino:', train_lin['ds'].min().date(), '->', train_lin['ds'].max().date())  # imprime intervalo de treino
print('Período teste :', test_lin['ds'].min().date(), '->', test_lin['ds'].max().date())  # imprime intervalo de teste


### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
baseline_value_lin = y_train_lin.iloc[-1]  # pega o último valor de y no treino
pred_baseline_lin = np.repeat(baseline_value_lin, len(y_test_lin))  # repete esse valor para todo o teste
mae_baseline_lin = mean_absolute_error(y_test_lin, pred_baseline_lin)  # calcula MAE do baseline
rmse_baseline_lin = np.sqrt(mean_squared_error(y_test_lin, pred_baseline_lin))  # calcula RMSE do baseline
print('Baseline MAE :', round(mae_baseline_lin, 3))  # imprime MAE
print('Baseline RMSE:', round(rmse_baseline_lin, 3))  # imprime RMSE


### 2.5 Modelos de regressão (execução linear)

Vamos treinar um modelo por vez, com uma célula de texto antes de cada treino.


#### Modelo: LinearRegression

**O que é:** regressão linear clássica; serve como baseline supervisionado simples.


In [ ]:
model_lr = LinearRegression()  # cria o modelo de regressão linear
model_lr.fit(X_train_lin, y_train_lin)  # treina o modelo
pred_lr = model_lr.predict(X_test_lin)  # gera previsão no teste
mae_lr = mean_absolute_error(y_test_lin, pred_lr)  # calcula MAE
rmse_lr = np.sqrt(mean_squared_error(y_test_lin, pred_lr))  # calcula RMSE
print('LinearRegression RMSE:', round(rmse_lr, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** ele entra na comparação final em condições iguais de treino/teste.


#### Modelo: Ridge

**O que é:** regressão linear com regularização L2; ajuda estabilidade dos coeficientes.


In [ ]:
model_ridge = Ridge(alpha=1.0, random_state=42)  # cria modelo Ridge
model_ridge.fit(X_train_lin, y_train_lin)  # treina o Ridge
pred_ridge = model_ridge.predict(X_test_lin)  # gera previsão no teste
mae_ridge = mean_absolute_error(y_test_lin, pred_ridge)  # calcula MAE
rmse_ridge = np.sqrt(mean_squared_error(y_test_lin, pred_ridge))  # calcula RMSE
print('Ridge RMSE:', round(rmse_ridge, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** ele entra na comparação final em condições iguais de treino/teste.


#### Modelo: ElasticNet

**O que é:** combina regularizações L1 e L2; útil com variáveis correlacionadas.


In [ ]:
model_en = ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42, max_iter=5000)  # cria modelo ElasticNet
model_en.fit(X_train_lin, y_train_lin)  # treina o ElasticNet
pred_en = model_en.predict(X_test_lin)  # gera previsão no teste
mae_en = mean_absolute_error(y_test_lin, pred_en)  # calcula MAE
rmse_en = np.sqrt(mean_squared_error(y_test_lin, pred_en))  # calcula RMSE
print('ElasticNet RMSE:', round(rmse_en, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** ele entra na comparação final em condições iguais de treino/teste.


#### Modelo: DecisionTree

**O que é:** captura não-linearidades por regras de divisão.


In [ ]:
model_tree = DecisionTreeRegressor(max_depth=4, random_state=42)  # cria modelo de árvore
model_tree.fit(X_train_lin, y_train_lin)  # treina árvore
pred_tree = model_tree.predict(X_test_lin)  # gera previsão no teste
mae_tree = mean_absolute_error(y_test_lin, pred_tree)  # calcula MAE
rmse_tree = np.sqrt(mean_squared_error(y_test_lin, pred_tree))  # calcula RMSE
print('DecisionTree RMSE:', round(rmse_tree, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** ele entra na comparação final em condições iguais de treino/teste.


#### Modelo: RandomForest

**O que é:** ensemble de árvores; tende a reduzir variância e melhorar generalização.


In [ ]:
model_rf = RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42)  # cria RandomForest
model_rf.fit(X_train_lin, y_train_lin)  # treina RandomForest
pred_rf = model_rf.predict(X_test_lin)  # gera previsão no teste
mae_rf = mean_absolute_error(y_test_lin, pred_rf)  # calcula MAE
rmse_rf = np.sqrt(mean_squared_error(y_test_lin, pred_rf))  # calcula RMSE
print('RandomForest RMSE:', round(rmse_rf, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** ele entra na comparação final em condições iguais de treino/teste.


#### Modelo: GradientBoosting

**O que é:** modelo de boosting que aprende correções sequenciais.


In [ ]:
model_gbm = GradientBoostingRegressor(random_state=42)  # cria modelo GradientBoosting
model_gbm.fit(X_train_lin, y_train_lin)  # treina GradientBoosting
pred_gbm = model_gbm.predict(X_test_lin)  # gera previsão no teste
mae_gbm = mean_absolute_error(y_test_lin, pred_gbm)  # calcula MAE
rmse_gbm = np.sqrt(mean_squared_error(y_test_lin, pred_gbm))  # calcula RMSE
print('GradientBoosting RMSE:', round(rmse_gbm, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** ele entra na comparação final em condições iguais de treino/teste.


---
### Teaser (ponte para Treino/Teste e comparação justa)
“OK, eu medi MAE/RMSE… mas em qual dado? E como eu sei se isso vai funcionar no próximo trimestre?”

Regras desta aula:
- Comparação justa: todos os modelos usam o MESMO split temporal (mesmo TESTE).
- Critério FP&A: escolhemos pelo desempenho no TESTE (MAE/RMSE), não pelo “passado perfeito”.
---


### 2.6 Tabela final e gráfico comparativo (PARTE A)


In [ ]:
results_a = pd.DataFrame([  # cria tabela de métricas da PARTE A
    {'modelo': 'Baseline_Naive', 'mae': mae_baseline_lin, 'rmse': rmse_baseline_lin},  # baseline
    {'modelo': 'LinearRegression', 'mae': mae_lr, 'rmse': rmse_lr},  # linear
    {'modelo': 'Ridge', 'mae': mae_ridge, 'rmse': rmse_ridge},  # ridge
    {'modelo': 'ElasticNet', 'mae': mae_en, 'rmse': rmse_en},  # elasticnet
    {'modelo': 'DecisionTree', 'mae': mae_tree, 'rmse': rmse_tree},  # árvore
    {'modelo': 'RandomForest', 'mae': mae_rf, 'rmse': rmse_rf},  # random forest
    {'modelo': 'GradientBoosting', 'mae': mae_gbm, 'rmse': rmse_gbm},  # gradient boosting
]).sort_values('rmse').reset_index(drop=True)  # ordena por melhor RMSE
results_a  # exibe ranking final


### Top 5 modelos (PARTE A) — leitura executiva rápida

Tabela interativa para ranking por RMSE e MAE no teste.

In [ ]:
rank_a = results_a.sort_values('rmse').head(5)  # seleciona top 5 modelos por RMSE na PARTE A
fig_table_a = go.Figure(data=[go.Table(  # cria tabela interativa de ranking da PARTE A
    header=dict(values=['Modelo', 'RMSE', 'MAE']),
    cells=dict(values=[rank_a['modelo'], rank_a['rmse'].round(3), rank_a['mae'].round(3)])
)])
fig_table_a.update_layout(title='PARTE A — Top 5 modelos por RMSE')  # define título da tabela
fig_table_a.show()  # exibe tabela interativa

### Leitura executiva em 20 segundos (PARTE A)

- A primeira linha da tabela é o melhor modelo no **TESTE**.
- Compare sempre com `Baseline_Naive`.
- O que importa para FP&A: erro menor e comportamento estável ao longo dos meses.


In [ ]:
best_a_exec = results_a.sort_values('rmse').iloc[0]  # captura melhor linha da PARTE A
rmse_rs_a = best_a_exec['rmse'] * 1_000_000  # converte RMSE de R$ milhões para R$
print(f"Melhor PARTE A: {best_a_exec['modelo']}")  # imprime modelo vencedor da PARTE A
print(f"RMSE no TESTE: {best_a_exec['rmse']:.3f} R$ mi (≈ R$ {rmse_rs_a:,.0f})")  # imprime RMSE em duas escalas
print(f"MAE no TESTE : {best_a_exec['mae']:.3f} R$ mi")  # imprime MAE do vencedor


### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
best_linear_name = results_a[results_a['modelo'].isin(['LinearRegression', 'Ridge', 'ElasticNet'])].iloc[0]['modelo']  # seleciona melhor linear
best_nonlinear_name = results_a[results_a['modelo'].isin(['DecisionTree', 'RandomForest', 'GradientBoosting'])].iloc[0]['modelo']  # seleciona melhor não-linear
pred_map_a = {  # dicionário com previsões da PARTE A
    'LinearRegression': pred_lr,
    'Ridge': pred_ridge,
    'ElasticNet': pred_en,
    'DecisionTree': pred_tree,
    'RandomForest': pred_rf,
    'GradientBoosting': pred_gbm,
}
plot_a_compare = pd.DataFrame({  # monta base comparativa do teste da PARTE A
    'ds': test_lin['ds'].values,
    'Real': y_test_lin.values,
    'Baseline_Naive': pred_baseline_lin,
    f'Melhor linear: {best_linear_name}': pred_map_a[best_linear_name],
    f'Melhor não-linear: {best_nonlinear_name}': pred_map_a[best_nonlinear_name],
})
plot_a_compare_long = plot_a_compare.melt(id_vars='ds', var_name='serie', value_name='valor')  # transforma para formato longo
fig_a_compare = px.line(plot_a_compare_long, x='ds', y='valor', color='serie', title='PARTE A - Real vs Previsto (teste)', labels={'ds': 'Data', 'valor': 'y (R$ milhões)', 'serie': 'Série'}, template='plotly_white')  # cria gráfico interativo
fig_a_compare.show()  # exibe comparação de curvas da PARTE A

**Pergunta para a turma (PARTE A):**
O modelo está errando mais nos picos, nos vales ou em mudanças rápidas de direção?


In [ ]:
best_model_a = results_a.sort_values('rmse').iloc[0]['modelo']  # identifica melhor modelo da PARTE A pelo menor RMSE
if best_model_a == 'Baseline_Naive':  # trata o caso em que o baseline seja o melhor
    pred_best_a = pred_baseline_lin  # usa previsão do baseline como melhor previsão
else:
    pred_best_a = pred_map_a.get(best_model_a, None)  # busca previsões do melhor modelo no dicionário

erro_a_df = pd.DataFrame({'ds': test_lin['ds'].values, 'Erro abs - Baseline_Naive': np.abs(y_test_lin - pred_baseline_lin)})  # cria série de erro absoluto do baseline
if pred_best_a is not None:
    erro_a_df[f'Erro abs - {best_model_a}'] = np.abs(y_test_lin - pred_best_a)  # adiciona série de erro do melhor modelo

erro_a_long = erro_a_df.melt(id_vars='ds', var_name='serie', value_name='erro_abs')  # converte para formato longo
fig_a_err = px.line(erro_a_long, x='ds', y='erro_abs', color='serie', title='PARTE A - Erro absoluto por mês (no TESTE)', labels={'ds': 'Mês', 'erro_abs': '|Erro| (unidade de y)', 'serie': 'Série'}, template='plotly_white')  # cria linha interativa
fig_a_err.show()  # exibe gráfico de erro absoluto da PARTE A
print('Leitura FP&A: picos de erro indicam meses com choque/sazonalidade/mudança de regime que merecem investigação.')  # interpretação de negócio

### Predito vs Real (PARTE A) com linha 45°

A linha 45° representa previsão perfeita; a distância dos pontos até ela representa erro.

In [ ]:
best_row_a = results_a[results_a['modelo'] == best_model_a].iloc[0]  # captura métricas do melhor modelo da PARTE A
rmse_a_best = float(best_row_a['rmse'])  # guarda RMSE para título executivo
mae_a_best = float(best_row_a['mae'])  # guarda MAE para título executivo

plot_a_pred = pd.DataFrame({'real': y_test_lin.values, 'predito': pred_best_a})  # monta base para scatter real vs predito
fig_a_pred = px.scatter(  # cria scatter interativo do teste
    plot_a_pred,
    x='real',
    y='predito',
    title=f'PARTE A — Predito vs Real (Teste) | {best_model_a} | RMSE={rmse_a_best:.2f} | MAE={mae_a_best:.2f}',
    labels={'real': 'Real (y)', 'predito': 'Predito (ŷ)'},
    template='plotly_white'
)
minv_a = float(min(plot_a_pred['real'].min(), plot_a_pred['predito'].min()))  # define menor valor para linha 45 graus
maxv_a = float(max(plot_a_pred['real'].max(), plot_a_pred['predito'].max()))  # define maior valor para linha 45 graus
fig_a_pred.add_trace(go.Scatter(x=[minv_a, maxv_a], y=[minv_a, maxv_a], mode='lines', name='Linha 45° (perfeito)'))  # adiciona referência y=x
fig_a_pred.show()  # renderiza gráfico executivo
print('Leitura FP&A: quanto mais perto da linha 45°, melhor o ajuste no período de teste.')  # mensagem didática

**Interpretação rápida do scatter (PARTE A):**
- Pontos **acima** da diagonal: modelo superestima.
- Pontos **abaixo** da diagonal: modelo subestima.


In [ ]:
resid_a = y_test_lin - pred_best_a  # calcula resíduos da PARTE A (Real - Predito)
resid_a_mean = float(np.mean(resid_a))  # calcula média dos resíduos para leitura rápida

fig_a_resid = px.histogram(  # cria histograma interativo dos resíduos
    x=resid_a,
    nbins=18,
    title=f'PARTE A — Resíduos no TESTE | {best_model_a} (Real - Predito)',
    labels={'x': 'Resíduo', 'count': 'Frequência'},
    template='plotly_white'
)
fig_a_resid.add_vline(x=0, line_dash='dash', line_color='red')  # adiciona linha vertical em zero como referência
fig_a_resid.add_annotation(x=0, y=1, xref='x', yref='paper', text=f'Média resíduo={resid_a_mean:.2f}', showarrow=False)  # anota média dos resíduos
fig_a_resid.show()  # exibe distribuição dos erros
print('Leitura FP&A: resíduos centrados em 0 e sem caudas extremas indicam melhor especificação.')  # interpretação de negócio

### Amostra de 12 meses do TESTE (PARTE A)

Tabela de auditoria mensal para discutir erros em sala.


In [ ]:
view_a = pd.DataFrame({  # cria tabela de auditoria da PARTE A no teste
    'ds': test_lin['ds'].values,  # coluna de data
    'real_y': y_test_lin.values,  # coluna de valor real
    'pred_y': pred_best_a,  # coluna de valor previsto
})  # fim da montagem inicial
view_a['erro_abs'] = np.abs(view_a['real_y'] - view_a['pred_y'])  # calcula erro absoluto por mês
view_a.head(12)  # exibe 12 linhas para leitura em sala


**Leitura FP&A:** esta tabela ajuda a auditar mês a mês onde o modelo acertou ou errou mais.


### 2.7 Feature importance (introdução)

### O que vamos fazer
Medir importância das variáveis para RandomForest e GradientBoosting.

### Por que isso importa para FP&A
Ajuda a priorizar quais drivers merecem atenção na discussão gerencial.


---
### Como a Feature Importance é calculada (explicação simples)

Em modelos de árvore (DecisionTree / RandomForest / GradientBoosting), a **feature importance** costuma ser calculada assim:

- Cada split do tipo “se X > valor” reduz o erro do modelo naquele nó.
- A importância de uma variável mede **quanto ela contribuiu para reduzir o erro** ao longo dos splits.
- No scikit-learn, a importância é a soma da **redução do critério** (ex.: MSE) atribuída à variável em todos os splits, agregada no conjunto de árvores (RF/GBM) e normalizada para somar 1.

✅ O que representa:
- Um **ranking de utilidade para previsão**: quais drivers mais ajudaram a diminuir o erro.

⚠️ O que NÃO representa:
- **Causalidade**. Importância alta não prova que “X causa Y”.
- Se duas variáveis são correlacionadas (ex.: Selic e spread), a importância pode “migrar” de uma para outra.

> Mensagem FP&A: use importance como bússola para priorizar drivers e conversas, e valide com lógica de negócio.
---


In [ ]:
rf_imp = pd.Series(model_rf.feature_importances_, index=features).sort_values(ascending=True)  # calcula importâncias do RandomForest
gbm_imp = pd.Series(model_gbm.feature_importances_, index=features).sort_values(ascending=True)  # calcula importâncias do GradientBoosting
rf_imp  # exibe vetor de importâncias do RandomForest


### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
rf_plot = rf_imp.sort_values(ascending=False).reset_index()  # ordena importâncias do RandomForest do maior para o menor
rf_plot.columns = ['feature', 'importancia']  # renomeia colunas para clareza

fig_rf_imp = px.bar(  # cria barras horizontais interativas do RandomForest
    rf_plot,
    x='importancia',
    y='feature',
    orientation='h',
    title='Importância de Variáveis — RandomForest',
    labels={'importancia': 'Importância', 'feature': 'Variável'},
    template='plotly_white'
)
fig_rf_imp.show()  # exibe gráfico de importance do RandomForest

### Importância no GBM

Mantemos o mesmo conceito de importance, agora em visual interativo para comparação rápida.

In [ ]:
gbm_plot = gbm_imp.sort_values(ascending=False).reset_index()  # ordena importâncias do GBM do maior para o menor
gbm_plot.columns = ['feature', 'importancia']  # renomeia colunas para clareza

fig_gbm_imp = px.bar(  # cria barras horizontais interativas do GradientBoosting
    gbm_plot,
    x='importancia',
    y='feature',
    orientation='h',
    title='Importância de Variáveis — GradientBoosting',
    labels={'importancia': 'Importância', 'feature': 'Variável'},
    template='plotly_white'
)
fig_gbm_imp.show()  # exibe gráfico de importance do GBM

✅ **Checkpoint A concluído:** já comparamos baseline e modelos em cenário linear e entendemos erros no TESTE.


## 3) PARTE B - Mesmo pipeline no dataset não-linear

### O que vamos fazer
Repetir o fluxo no dataset não-linear.

### Por que isso importa para FP&A
Em cenários com limiares/interações, modelos não-lineares tendem a capturar melhor os padrões.


### 3.1 Carregar base não-linear e visualizar padrão


In [ ]:
url_nlin = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_regressao_nonlinear.csv'  # URL da base não-linear
df_nlin = pd.read_csv(url_nlin)  # leitura da base não-linear
df_nlin['ds'] = pd.to_datetime(df_nlin['ds'])  # conversão da data para datetime
df_nlin.head()  # exibe primeiras linhas


### Série temporal inicial (PARTE B)

Primeiro visualizamos y no tempo para entender o padrão geral antes dos modelos.

In [ ]:
fig_b_line = px.line(  # cria gráfico de linha interativo da base não-linear
    df_nlin,
    x='ds',
    y='y',
    title='PARTE B — Originação (y) ao longo do tempo',
    labels={'ds': 'Data', 'y': 'y (R$ milhões)'},
    template='plotly_white'
)
fig_b_line.show()  # exibe série temporal da PARTE B

---
### Visualizando o “regime” não-linear (por que árvores podem ganhar aqui)
No dataset não-linear, existe um comportamento por faixas (limiar). Vamos marcar um “regime” para visualizar onde o padrão muda.
---


In [ ]:
threshold = 9.5  # define limiar de SELIC para separar regimes
df_nl = df_nlin.copy()  # cria cópia da base não-linear para análise de regime
df_nl['regime_selic'] = np.where(df_nl['selic'] <= threshold, f'Selic <= {threshold}', f'Selic > {threshold}')  # cria rótulo textual de regime

fig_regime = px.scatter(  # cria dispersão interativa de Selic vs y por regime
    df_nl,
    x='selic',
    y='y',
    color='regime_selic',
    hover_data=['selic', 'y', 'regime_selic'],
    title='PARTE B — Selic vs y (regimes) — por que árvores podem ganhar',
    labels={'selic': 'Selic', 'y': 'y (originação)'},
    template='plotly_white'
)
fig_regime.show()  # exibe gráfico de regime não-linear
print('Leitura FP&A: padrões por faixa indicam que uma reta única tende a errar mais que árvores/GBM.')  # interpretação executiva

### 3.2 Split temporal e baseline (PARTE B)


In [ ]:
split_idx_b = int(len(df_nlin) * 0.8)  # define ponto de corte de treino/teste
train_nlin = df_nlin.iloc[:split_idx_b].copy()  # recorte de treino da base não-linear
train_nlin = train_nlin.sort_values('ds')  # garante ordenação temporal de treino
test_nlin = df_nlin.iloc[split_idx_b:].copy()  # recorte de teste da base não-linear
test_nlin = test_nlin.sort_values('ds')  # garante ordenação temporal de teste
X_train_nlin = train_nlin[features]  # monta X de treino
X_test_nlin = test_nlin[features]  # monta X de teste
y_train_nlin = train_nlin['y']  # monta y de treino
y_test_nlin = test_nlin['y']  # monta y de teste

# aliases para facilitar leitura nos gráficos extras solicitados
X_train_nl = X_train_nlin  # alias do X treino
X_test_nl = X_test_nlin  # alias do X teste
y_train_nl = y_train_nlin  # alias do y treino
y_test_nl = y_test_nlin  # alias do y teste
test_nl = test_nlin  # alias do DataFrame de teste

baseline_value_nlin = y_train_nlin.iloc[-1]  # captura último valor de treino para baseline
pred_baseline_nlin = np.repeat(baseline_value_nlin, len(y_test_nlin))  # gera previsão baseline
pred_baseline_nl = pred_baseline_nlin  # alias da previsão baseline
mae_baseline_nlin = mean_absolute_error(y_test_nlin, pred_baseline_nlin)  # calcula MAE do baseline
rmse_baseline_nlin = np.sqrt(mean_squared_error(y_test_nlin, pred_baseline_nlin))  # calcula RMSE do baseline
print('Baseline RMSE (não-linear):', round(rmse_baseline_nlin, 3))  # imprime RMSE baseline da PARTE B


---
### Mini-experimento: profundidade da árvore e overfitting (treino vs teste)
Uma árvore muito profunda pode “decorar” o histórico. Vamos ver o efeito da profundidade no erro de treino e teste.
---


In [ ]:
depths = list(range(1, 16))  # define profundidades a testar
rmse_train, rmse_test = [], []  # inicializa listas de erro de treino e teste
for d in depths:  # testa cada profundidade candidata
    m = DecisionTreeRegressor(max_depth=d, random_state=42)  # cria árvore com profundidade d
    m.fit(X_train_nl, y_train_nl)  # treina árvore no conjunto de treino
    pred_tr = m.predict(X_train_nl)  # prevê no treino
    pred_te = m.predict(X_test_nl)  # prevê no teste
    rmse_train.append(np.sqrt(mean_squared_error(y_train_nl, pred_tr)))  # guarda RMSE de treino
    rmse_test.append(np.sqrt(mean_squared_error(y_test_nl, pred_te)))  # guarda RMSE de teste
overfit_df = pd.DataFrame({'max_depth': depths, 'RMSE Treino': rmse_train, 'RMSE Teste': rmse_test})  # organiza resultados em tabela
overfit_long = overfit_df.melt(id_vars='max_depth', var_name='serie', value_name='rmse')  # converte para formato longo
fig_overfit = px.line(overfit_long, x='max_depth', y='rmse', color='serie', markers=True, title='PARTE B — Overfitting em árvore: RMSE vs max_depth', labels={'max_depth': 'max_depth', 'rmse': 'RMSE', 'serie': 'Série'}, template='plotly_white')  # cria gráfico interativo
fig_overfit.show()  # renderiza gráfico treino vs teste
print('Leitura FP&A: treino cai e teste sobe indica overfitting (modelo decorou o passado).')  # interpretação de negócio

### 3.3 Treinar modelos na base não-linear

Vamos repetir os mesmos modelos da PARTE A para comparação justa.


#### Modelo: LinearRegression (PARTE B)


In [ ]:
model_lr_b = LinearRegression()  # cria LinearRegression da PARTE B
model_lr_b.fit(X_train_nlin, y_train_nlin)  # treina LinearRegression na base não-linear
pred_lr_b = model_lr_b.predict(X_test_nlin)  # gera previsão da LinearRegression


#### Modelo: Ridge (PARTE B)


In [ ]:
model_ridge_b = Ridge(alpha=1.0, random_state=42)  # cria Ridge da PARTE B
model_ridge_b.fit(X_train_nlin, y_train_nlin)  # treina Ridge na base não-linear
pred_ridge_b = model_ridge_b.predict(X_test_nlin)  # gera previsão do Ridge


#### Modelo: ElasticNet (PARTE B)


In [ ]:
model_en_b = ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42, max_iter=5000)  # cria ElasticNet da PARTE B
model_en_b.fit(X_train_nlin, y_train_nlin)  # treina ElasticNet na base não-linear
pred_en_b = model_en_b.predict(X_test_nlin)  # gera previsão do ElasticNet


#### Modelo: DecisionTree (PARTE B)


In [ ]:
model_tree_b = DecisionTreeRegressor(max_depth=4, random_state=42)  # cria árvore da PARTE B
model_tree_b.fit(X_train_nlin, y_train_nlin)  # treina árvore da PARTE B
pred_tree_b = model_tree_b.predict(X_test_nlin)  # gera previsão da árvore


#### Modelo: RandomForest (PARTE B)


In [ ]:
model_rf_b = RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42)  # cria RandomForest da PARTE B
model_rf_b.fit(X_train_nlin, y_train_nlin)  # treina RandomForest da PARTE B
pred_rf_b = model_rf_b.predict(X_test_nlin)  # gera previsão do RandomForest


#### Modelo: GradientBoosting (PARTE B)


In [ ]:
model_gbm_b = GradientBoostingRegressor(random_state=42)  # cria GradientBoosting da PARTE B
model_gbm_b.fit(X_train_nlin, y_train_nlin)  # treina GradientBoosting da PARTE B
pred_gbm_b = model_gbm_b.predict(X_test_nlin)  # gera previsão do GradientBoosting


### 3.4 Tabela de métricas (PARTE B)

Agora comparamos MAE/RMSE de todos os modelos no mesmo TESTE da PARTE B.


**Se aparecer `NameError` aqui:** significa que alguma célula de treino da PARTE B não rodou. Volte e execute a seção de treino.


In [ ]:
results_b = pd.DataFrame([  # cria tabela de resultados da PARTE B
    {'modelo': 'Baseline_Naive', 'mae': mae_baseline_nlin, 'rmse': rmse_baseline_nlin},  # linha baseline
    {'modelo': 'LinearRegression', 'mae': mean_absolute_error(y_test_nlin, pred_lr_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_lr_b))},  # linha linear
    {'modelo': 'Ridge', 'mae': mean_absolute_error(y_test_nlin, pred_ridge_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_ridge_b))},  # linha ridge
    {'modelo': 'ElasticNet', 'mae': mean_absolute_error(y_test_nlin, pred_en_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_en_b))},  # linha elasticnet
    {'modelo': 'DecisionTree', 'mae': mean_absolute_error(y_test_nlin, pred_tree_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_tree_b))},  # linha árvore
    {'modelo': 'RandomForest', 'mae': mean_absolute_error(y_test_nlin, pred_rf_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_rf_b))},  # linha random forest
    {'modelo': 'GradientBoosting', 'mae': mean_absolute_error(y_test_nlin, pred_gbm_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_gbm_b))},  # linha gradient boosting
]).sort_values('rmse').reset_index(drop=True)  # ordena por melhor RMSE
results_b  # exibe ranking final da PARTE B


### Top 5 modelos (PARTE B) — leitura executiva rápida

Ranking por RMSE para comparar comportamento em dado não-linear.

In [ ]:
rank_b = results_b.sort_values('rmse').head(5)  # seleciona top 5 modelos por RMSE na PARTE B
fig_table_b = go.Figure(data=[go.Table(  # cria tabela interativa de ranking da PARTE B
    header=dict(values=['Modelo', 'RMSE', 'MAE']),
    cells=dict(values=[rank_b['modelo'], rank_b['rmse'].round(3), rank_b['mae'].round(3)])
)])
fig_table_b.update_layout(title='PARTE B — Top 5 modelos por RMSE')  # define título da tabela
fig_table_b.show()  # exibe tabela interativa

### Leitura executiva em 20 segundos (PARTE B)

- Observe se o vencedor mudou em relação à PARTE A.
- Em dados com regime, modelos não-lineares costumam subir no ranking.
- Decisão de negócio: escolher o modelo mais robusto no TESTE.


In [ ]:
best_b_exec = results_b.sort_values('rmse').iloc[0]  # captura melhor linha da PARTE B
rmse_rs_b = best_b_exec['rmse'] * 1_000_000  # converte RMSE de R$ milhões para R$
print(f"Melhor PARTE B: {best_b_exec['modelo']}")  # imprime modelo vencedor da PARTE B
print(f"RMSE no TESTE: {best_b_exec['rmse']:.3f} R$ mi (≈ R$ {rmse_rs_b:,.0f})")  # imprime RMSE em duas escalas
print(f"MAE no TESTE : {best_b_exec['mae']:.3f} R$ mi")  # imprime MAE do vencedor


### 3.5 Gráfico comparativo (PARTE B)

Vamos comparar visualmente o melhor modelo linear versus o melhor não-linear.


In [ ]:
best_linear_b = results_b[results_b['modelo'].isin(['LinearRegression', 'Ridge', 'ElasticNet'])].iloc[0]['modelo']  # identifica melhor linear da PARTE B
best_nonlinear_b = results_b[results_b['modelo'].isin(['DecisionTree', 'RandomForest', 'GradientBoosting'])].iloc[0]['modelo']  # identifica melhor não-linear da PARTE B
pred_map_b = {  # cria dicionário de previsões da PARTE B
    'LinearRegression': pred_lr_b,
    'Ridge': pred_ridge_b,
    'ElasticNet': pred_en_b,
    'DecisionTree': pred_tree_b,
    'RandomForest': pred_rf_b,
    'GradientBoosting': pred_gbm_b,
}
plot_b_compare = pd.DataFrame({  # consolida séries para comparação visual da PARTE B
    'ds': test_nlin['ds'].values,
    'Real': y_test_nlin.values,
    f'Melhor linear: {best_linear_b}': pred_map_b[best_linear_b],
    f'Melhor não-linear: {best_nonlinear_b}': pred_map_b[best_nonlinear_b],
})
plot_b_compare_long = plot_b_compare.melt(id_vars='ds', var_name='serie', value_name='valor')  # transforma para formato longo
fig_b_compare = px.line(plot_b_compare_long, x='ds', y='valor', color='serie', title='PARTE B - Melhor linear vs melhor não-linear (teste)', labels={'ds': 'Data', 'valor': 'y (R$ milhões)', 'serie': 'Série'}, template='plotly_white')  # cria gráfico interativo
fig_b_compare.show()  # exibe comparação da PARTE B

**Pergunta para a turma (PARTE B):**
Em quais meses o modelo não-linear claramente melhora em relação ao linear?


In [ ]:
best_model_b = results_b.sort_values('rmse').iloc[0]['modelo']  # identifica melhor modelo da PARTE B pelo menor RMSE
if best_model_b == 'Baseline_Naive':
    pred_best_b = pred_baseline_nlin  # usa baseline quando ele for o vencedor
else:
    pred_best_b = pred_map_b.get(best_model_b, None)  # busca previsão do melhor modelo da PARTE B

erro_b_df = pd.DataFrame({'ds': test_nl['ds'].values, 'Erro abs - Baseline_Naive': np.abs(y_test_nl - pred_baseline_nl)})  # monta série de erro baseline
if pred_best_b is not None:
    erro_b_df[f'Erro abs - {best_model_b}'] = np.abs(y_test_nl - pred_best_b)  # inclui erro do modelo vencedor

erro_b_long = erro_b_df.melt(id_vars='ds', var_name='serie', value_name='erro_abs')  # converte para formato longo
fig_b_err = px.line(erro_b_long, x='ds', y='erro_abs', color='serie', title='PARTE B - Erro absoluto por mês (no TESTE)', labels={'ds': 'Mês', 'erro_abs': '|Erro| (unidade de y)', 'serie': 'Série'}, template='plotly_white')  # cria linha interativa
fig_b_err.show()  # exibe erro absoluto na PARTE B

### Predito vs Real (PARTE B) com linha 45°

Repetimos o mesmo padrão da PARTE A para reduzir carga cognitiva na interpretação.

In [ ]:
best_row_b = results_b[results_b['modelo'] == best_model_b].iloc[0]  # captura métricas do melhor modelo da PARTE B
rmse_b_best = float(best_row_b['rmse'])  # guarda RMSE para título executivo
mae_b_best = float(best_row_b['mae'])  # guarda MAE para título executivo

plot_b_pred = pd.DataFrame({'real': y_test_nl.values, 'predito': pred_best_b})  # monta base para scatter real vs predito
fig_b_pred = px.scatter(  # cria scatter interativo da PARTE B
    plot_b_pred,
    x='real',
    y='predito',
    title=f'PARTE B — Predito vs Real (Teste) | {best_model_b} | RMSE={rmse_b_best:.2f} | MAE={mae_b_best:.2f}',
    labels={'real': 'Real (y)', 'predito': 'Predito (ŷ)'},
    template='plotly_white'
)
minv_b = float(min(plot_b_pred['real'].min(), plot_b_pred['predito'].min()))  # define mínimo para linha 45 graus
maxv_b = float(max(plot_b_pred['real'].max(), plot_b_pred['predito'].max()))  # define máximo para linha 45 graus
fig_b_pred.add_trace(go.Scatter(x=[minv_b, maxv_b], y=[minv_b, maxv_b], mode='lines', name='Linha 45° (perfeito)'))  # adiciona referência y=x
fig_b_pred.show()  # exibe scatter interativo da PARTE B

**Interpretação rápida do scatter (PARTE B):**
- Pontos próximos da diagonal: bom ajuste no teste.
- Desvios sistemáticos para um lado indicam viés.


In [ ]:
resid_b = y_test_nl - pred_best_b  # calcula resíduos da PARTE B (Real - Predito)
resid_b_mean = float(np.mean(resid_b))  # calcula média dos resíduos da PARTE B

fig_b_resid = px.histogram(  # cria histograma interativo dos resíduos da PARTE B
    x=resid_b,
    nbins=18,
    title=f'PARTE B — Resíduos no TESTE | {best_model_b} (Real - Predito)',
    labels={'x': 'Resíduo', 'count': 'Frequência'},
    template='plotly_white'
)
fig_b_resid.add_vline(x=0, line_dash='dash', line_color='red')  # adiciona linha de referência em zero
fig_b_resid.add_annotation(x=0, y=1, xref='x', yref='paper', text=f'Média resíduo={resid_b_mean:.2f}', showarrow=False)  # anota média dos resíduos
fig_b_resid.show()  # exibe distribuição de resíduos da PARTE B

**Leitura dos resíduos (PARTE B):**
Resíduos muito assimétricos ou com caudas longas indicam potencial de melhoria de modelo/drivers.


In [ ]:
view_b = pd.DataFrame({  # cria tabela de auditoria da PARTE B no teste
    'ds': test_nl['ds'].values,  # coluna de data
    'real_y': y_test_nl.values,  # coluna de valor real
    'pred_y': pred_best_b,  # coluna de valor previsto
})  # fim da montagem inicial
view_b['erro_abs'] = np.abs(view_b['real_y'] - view_b['pred_y'])  # calcula erro absoluto por mês
view_b.head(12)  # exibe 12 linhas para leitura em sala


**Amostra de 12 meses do TESTE (PARTE B):** use para discutir erros mês a mês com a turma.


### 3.6 Visão executiva: ranking muda quando o dado muda

O objetivo aqui é comparar os **mesmos modelos** nos dois cenários:
- PARTE A (dataset linear)
- PARTE B (dataset não-linear)

Mensagem-chave: **não existe campeão universal**. O desempenho depende do padrão do dado.


In [ ]:
comp_a = results_a[['modelo', 'rmse']].copy()  # seleciona colunas de modelo e RMSE da PARTE A
comp_b = results_b[['modelo', 'rmse']].copy()  # seleciona colunas de modelo e RMSE da PARTE B
comp = comp_a.merge(comp_b, on='modelo', suffixes=('_A_linear', '_B_nlinear'))  # junta resultados A e B por modelo
comp = comp.sort_values('rmse_B_nlinear').reset_index(drop=True)  # ordena pelo desempenho da PARTE B

comp_long = comp.melt(id_vars='modelo', value_vars=['rmse_A_linear', 'rmse_B_nlinear'], var_name='serie', value_name='rmse')  # converte para formato longo
comp_long['serie'] = comp_long['serie'].map({'rmse_A_linear': 'RMSE PARTE A (linear)', 'rmse_B_nlinear': 'RMSE PARTE B (não-linear)'})  # traduz rótulos
fig_comp_rmse = px.bar(comp_long, x='modelo', y='rmse', color='serie', barmode='group', title='Comparativo executivo de RMSE no TESTE: PARTE A vs PARTE B', labels={'modelo': 'Modelo', 'rmse': 'RMSE', 'serie': 'Série'}, template='plotly_white')  # cria barras agrupadas
fig_comp_rmse.show()  # renderiza comparativo de RMSE
print('Leitura executiva: o ranking muda quando o padrão do dado muda; escolha de modelo deve ser contextual.')  # conclusão executiva

### 3.7 Cartões de conclusão: vencedor por parte


In [ ]:
best_a = results_a.sort_values('rmse').iloc[0]  # captura melhor modelo da PARTE A
best_b = results_b.sort_values('rmse').iloc[0]  # captura melhor modelo da PARTE B

reason_map_a = {  # justificativas executivas para vencedor da PARTE A
    'LinearRegression': 'venceu por simplicidade e boa aderência em padrão mais linear.',
    'Ridge': 'venceu por estabilizar coeficientes e reduzir sensibilidade.',
    'ElasticNet': 'venceu por equilibrar regularização L1/L2.',
    'DecisionTree': 'venceu por capturar regras e não-linearidades locais.',
    'RandomForest': 'venceu por robustez em relações não-lineares.',
    'GradientBoosting': 'venceu por capturar padrões complexos com menor erro no teste.',
    'Baseline_Naive': 'venceu porque os modelos não superaram uma referência simples.'
}
reason_map_b = {  # justificativas executivas para vencedor da PARTE B
    'LinearRegression': 'venceu mesmo em base não-linear, indicando relação ainda parcialmente estável.',
    'Ridge': 'venceu por bom equilíbrio entre viés e variância.',
    'ElasticNet': 'venceu por regularização eficiente no cenário não-linear.',
    'DecisionTree': 'venceu por capturar limiares e regras de regime.',
    'RandomForest': 'venceu por capturar interações e reduzir variância.',
    'GradientBoosting': 'venceu por modelar não-linearidades de forma incremental.',
    'Baseline_Naive': 'venceu porque os modelos não superaram a referência simples.'
}

### Leitura dos cartões de vencedor

Nesta célula vamos apenas imprimir o resumo final (modelo + métricas + justificativa) para cada parte.

In [ ]:
print('PARTE A - Vencedor no TESTE')  # imprime título do cartão da PARTE A
print(f"Modelo: {best_a['modelo']} | RMSE: {best_a['rmse']:.3f} | MAE: {best_a['mae']:.3f}")  # imprime métricas da PARTE A
print('Por que venceu:', reason_map_a.get(best_a['modelo'], 'desempenho superior no período de teste.'))  # imprime justificativa A
print('-' * 90)  # imprime separador visual
print('PARTE B - Vencedor no TESTE')  # imprime título do cartão da PARTE B
print(f"Modelo: {best_b['modelo']} | RMSE: {best_b['rmse']:.3f} | MAE: {best_b['mae']:.3f}")  # imprime métricas da PARTE B
print('Por que venceu:', reason_map_b.get(best_b['modelo'], 'desempenho superior no período de teste.'))  # imprime justificativa B

### 3.8 Top 3 por parte + recomendação prática


In [ ]:
top3_a = results_a.sort_values('rmse').head(3).copy()  # seleciona top 3 modelos da PARTE A por RMSE
top3_b = results_b.sort_values('rmse').head(3).copy()  # seleciona top 3 modelos da PARTE B por RMSE

print('Top 3 - PARTE A (RMSE no TESTE)')  # título da lista A
display(top3_a)  # exibe top 3 da PARTE A
print('Top 3 - PARTE B (RMSE no TESTE)')  # título da lista B
display(top3_b)  # exibe top 3 da PARTE B

print('Recomendação prática: sempre manter baseline + 1 candidato final; validar estabilidade.')  # recomendação operacional


### 3.9 Como a importance muda quando o dado muda (PARTE A vs PARTE B)

Aqui comparamos importâncias do mesmo tipo de modelo de árvore (RandomForest) nos dois cenários.


### 3.8.1 Tabela Champion vs Challenger (operacional)

Sugestão para governança: manter um modelo principal e um desafiador em monitoramento contínuo.


In [ ]:
champ_a = results_a.sort_values('rmse').iloc[0]  # define champion da PARTE A
chall_a = results_a.sort_values('rmse').iloc[1]  # define challenger da PARTE A
champ_b = results_b.sort_values('rmse').iloc[0]  # define champion da PARTE B
chall_b = results_b.sort_values('rmse').iloc[1]  # define challenger da PARTE B

governanca = pd.DataFrame([  # monta quadro operacional de governança
    {'parte': 'A (linear)', 'champion': champ_a['modelo'], 'challenger': chall_a['modelo'], 'metrica_alvo': 'RMSE TESTE', 'cadencia_retreino': 'Mensal'},
    {'parte': 'B (não-linear)', 'champion': champ_b['modelo'], 'challenger': chall_b['modelo'], 'metrica_alvo': 'RMSE TESTE', 'cadencia_retreino': 'Mensal'},
])  # fim do quadro

governanca  # exibe tabela champion/challenger


### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
imp_a = pd.Series(model_rf.feature_importances_, index=features)  # extrai importâncias do RandomForest da PARTE A
imp_b = pd.Series(model_rf_b.feature_importances_, index=features)  # extrai importâncias do RandomForest da PARTE B
imp_comp = pd.DataFrame({'A_linear': imp_a, 'B_nlinear': imp_b})  # monta tabela comparativa de importâncias
imp_comp['media'] = imp_comp.mean(axis=1)  # calcula média para escolher variáveis mais relevantes
imp_top6 = imp_comp.sort_values('media', ascending=False).head(6).drop(columns='media').reset_index()  # seleciona top 6 features
imp_top6 = imp_top6.rename(columns={'index': 'feature'})  # renomeia coluna de índice para feature

imp_long = imp_top6.melt(id_vars='feature', var_name='serie', value_name='importancia')  # converte para formato longo
imp_long['serie'] = imp_long['serie'].map({'A_linear': 'PARTE A (linear)', 'B_nlinear': 'PARTE B (não-linear)'})  # traduz nomes das séries
fig_imp_shift = px.bar(imp_long, x='importancia', y='feature', color='serie', orientation='h', barmode='group', title='Top 6 features: mudança de importância (A vs B)', labels={'importancia': 'Importância relativa', 'feature': 'Variável', 'serie': 'Série'}, template='plotly_white')  # cria barras horizontais agrupadas
fig_imp_shift.show()  # exibe mudança de importance entre as partes
print('Leitura FP&A: importance é sensível a regimes; correlação entre drivers pode mudar o ranking.')  # interpretação de negócio

### 3.10 Checklist FP&A (1 minuto)

- baseline obrigatório
- split temporal
- métricas no teste (MAE/RMSE)
- checar overfitting (treino vs teste)
- interpretabilidade (importance/regras) com cuidado
- monitoramento + re-treino em mudança de regime


✅ **Checkpoint B concluído:** agora você viu como regimes não-lineares mudam ranking, erros e importance dos modelos.


---
## Ponte final para PARTE C: quando escolher t→Y

Quando séries temporais são mais adequadas:
- quando o histórico de `y` já mostra padrão forte de tendência/sazonalidade;
- quando os drivers explicativos são limitados ou instáveis;
- quando o objetivo é forecast recorrente de curto/médio prazo.

Casos típicos em FP&A de banco de varejo:
- rolling forecast mensal de originação;
- revisão de orçamento trimestral;
- acompanhamento de metas com atualização contínua.

Na PARTE C vamos comparar:
- ETS como referência temporal;
- ARIMAX com Selic como exógena;
- Prophet com eventos/regressores (opcional).

Regra de ouro continua a mesma: validar no tempo (treino no passado, teste no futuro).
---


## 4) PARTE C - Séries temporais (t -> y)

### O que vamos fazer
Comparar baseline, ETS, ARIMA, ARIMAX e Prophet (opcional).

### Por que isso importa para FP&A
Muitas previsões corporativas são feitas a partir do histórico da própria série.


In [ ]:
url_ts = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_timeseries.csv'  # URL da base temporal
df_ts = pd.read_csv(url_ts)  # leitura da base temporal
df_ts['ds'] = pd.to_datetime(df_ts['ds'])  # conversão da data para datetime
df_ts.head()  # exibe primeiras linhas


### Série temporal da PARTE C (Plotly)

Aqui entramos no bloco t→y com visual interativo para leitura de tendência e sazonalidade.

In [ ]:
fig_c_line = px.line(  # cria série temporal interativa da PARTE C
    df_ts,
    x='ds',
    y='y',
    title='PARTE C — Série temporal y',
    labels={'ds': 'Data', 'y': 'y (R$ milhões)'},
    template='plotly_white'
)
fig_c_line.show()  # exibe série temporal de entrada

### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
holdout = 12  # define últimos 12 meses como teste
train_ts = df_ts.iloc[:-holdout].copy()  # separa treino temporal
test_ts = df_ts.iloc[-holdout:].copy()  # separa teste temporal
y_train_ts = train_ts['y'].values  # vetor y de treino
y_test_ts = test_ts['y'].values  # vetor y de teste
print('Treino:', train_ts.shape, 'Teste:', test_ts.shape)  # imprime tamanhos
print('Período treino:', train_ts['ds'].min().date(), '->', train_ts['ds'].max().date())  # imprime período de treino
print('Período teste :', test_ts['ds'].min().date(), '->', test_ts['ds'].max().date())  # imprime período de teste


### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
pred_naive_ts = np.repeat(y_train_ts[-1], len(y_test_ts))  # cria previsão naive do período de teste
mae_naive_ts = mean_absolute_error(y_test_ts, pred_naive_ts)  # calcula MAE baseline temporal
rmse_naive_ts = np.sqrt(mean_squared_error(y_test_ts, pred_naive_ts))  # calcula RMSE baseline temporal
print('Baseline MAE :', round(mae_naive_ts, 3))  # imprime MAE baseline temporal
print('Baseline RMSE:', round(rmse_naive_ts, 3))  # imprime RMSE baseline temporal


### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
ets = ExponentialSmoothing(train_ts['y'], trend='add', seasonal='add', seasonal_periods=12).fit(optimized=True)  # ajusta ETS
pred_ets = ets.forecast(len(test_ts)).values  # prevê com ETS para período de teste
mae_ets = mean_absolute_error(y_test_ts, pred_ets)  # calcula MAE ETS
rmse_ets = np.sqrt(mean_squared_error(y_test_ts, pred_ets))  # calcula RMSE ETS
print('ETS MAE :', round(mae_ets, 3))  # imprime MAE ETS
print('ETS RMSE:', round(rmse_ets, 3))  # imprime RMSE ETS


### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
arima = SARIMAX(train_ts['y'], order=(1, 1, 1), enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)  # ajusta ARIMA
pred_arima = arima.forecast(steps=len(test_ts)).values  # prevê com ARIMA no período de teste
mae_arima = mean_absolute_error(y_test_ts, pred_arima)  # calcula MAE ARIMA
rmse_arima = np.sqrt(mean_squared_error(y_test_ts, pred_arima))  # calcula RMSE ARIMA
print('ARIMA MAE :', round(mae_arima, 3))  # imprime MAE ARIMA
print('ARIMA RMSE:', round(rmse_arima, 3))  # imprime RMSE ARIMA


### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
arimax = SARIMAX(train_ts['y'], exog=train_ts[['selic']], order=(1, 1, 1), enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)  # ajusta ARIMAX com selic
pred_arimax = arimax.forecast(steps=len(test_ts), exog=test_ts[['selic']]).values  # prevê com ARIMAX usando selic no teste
mae_arimax = mean_absolute_error(y_test_ts, pred_arimax)  # calcula MAE ARIMAX
rmse_arimax = np.sqrt(mean_squared_error(y_test_ts, pred_arimax))  # calcula RMSE ARIMAX
print('ARIMAX MAE :', round(mae_arimax, 3))  # imprime MAE ARIMAX
print('ARIMAX RMSE:', round(rmse_arimax, 3))  # imprime RMSE ARIMAX


### 4.1 Prophet (opcional)

Se não instalar, siga a aula normalmente com ETS/ARIMA/ARIMAX.


In [ ]:
import importlib.util  # utilitário para verificar se pacote está disponível sem bloco de exceção

prophet_ok = importlib.util.find_spec('prophet') is not None  # verifica disponibilidade do pacote prophet
if prophet_ok:  # executa import apenas quando prophet estiver instalado
    from prophet import Prophet  # importa classe Prophet para o bloco opcional
    print('Prophet disponível nesta sessão.')  # confirma disponibilidade do modelo
else:  # segue fluxo sem erro quando prophet não está instalado
    print('Prophet não instalado; bloco opcional será pulado.')  # informa pulo do bloco opcional

### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
pred_prophet = None  # inicializa vetor de previsão Prophet
mae_prophet = None  # inicializa MAE Prophet
rmse_prophet = None  # inicializa RMSE Prophet
if prophet_ok:  # executa bloco somente se Prophet disponível
    train_prophet = train_ts[['ds', 'y', 'selic', 'evento']].copy()  # monta base de treino no formato Prophet
    test_prophet = test_ts[['ds', 'selic', 'evento']].copy()  # monta base de teste no formato Prophet
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)  # cria modelo Prophet
    m.add_regressor('selic')  # adiciona regressor selic
    m.add_regressor('evento')  # adiciona regressor evento
    m.fit(train_prophet)  # treina Prophet
    fcst = m.predict(test_prophet)  # prevê no período de teste
    pred_prophet = fcst['yhat'].values  # extrai vetor de previsão
    mae_prophet = mean_absolute_error(y_test_ts, pred_prophet)  # calcula MAE Prophet
    rmse_prophet = np.sqrt(mean_squared_error(y_test_ts, pred_prophet))  # calcula RMSE Prophet
    print('Prophet MAE :', round(mae_prophet, 3))  # imprime MAE Prophet
    print('Prophet RMSE:', round(rmse_prophet, 3))  # imprime RMSE Prophet
else:  # executa quando Prophet indisponível
    print('Prophet não executado.')  # informa que etapa foi pulada


### Próximo passo

### O que vamos fazer
Executar a próxima etapa do pipeline de forma incremental.

### Por que isso importa para FP&A
Blocos curtos facilitam auditoria e entendimento do impacto de cada comando.

### O que observar no output
Confira a métrica/tabela gerada antes de seguir para a próxima célula.

In [ ]:
results_c = pd.DataFrame([  # cria tabela inicial da PARTE C
    {'modelo': 'Baseline_Naive', 'mae': mae_naive_ts, 'rmse': rmse_naive_ts},  # linha baseline
    {'modelo': 'ETS_HoltWinters', 'mae': mae_ets, 'rmse': rmse_ets},  # linha ETS
    {'modelo': 'ARIMA', 'mae': mae_arima, 'rmse': rmse_arima},  # linha ARIMA
    {'modelo': 'ARIMAX_selic', 'mae': mae_arimax, 'rmse': rmse_arimax},  # linha ARIMAX
])  # fecha DataFrame inicial
if pred_prophet is not None:  # verifica se Prophet está disponível
    results_c = pd.concat([results_c, pd.DataFrame([{'modelo': 'Prophet', 'mae': mae_prophet, 'rmse': rmse_prophet}])], ignore_index=True)  # acrescenta linha Prophet
results_c = results_c.sort_values('rmse').reset_index(drop=True)  # ordena por melhor RMSE
results_c  # exibe tabela final da PARTE C


### Top 5 modelos (PARTE C) — leitura executiva rápida

Ranking por RMSE para o bloco de séries temporais.

In [ ]:
rank_c = results_c.sort_values('rmse').head(5)  # seleciona top 5 modelos por RMSE na PARTE C
fig_table_c = go.Figure(data=[go.Table(  # cria tabela interativa de ranking da PARTE C
    header=dict(values=['Modelo', 'RMSE', 'MAE']),
    cells=dict(values=[rank_c['modelo'], rank_c['rmse'].round(3), rank_c['mae'].round(3)])
)])
fig_table_c.update_layout(title='PARTE C — Top 5 modelos por RMSE')  # define título da tabela
fig_table_c.show()  # exibe tabela interativa

### Leitura executiva em 20 segundos (PARTE C)

- Veja qual modelo temporal ganhou no TESTE.
- Compare o ganho versus baseline temporal.
- Traduza RMSE para ordem de grandeza financeira mensal.


In [ ]:
best_c_exec = results_c.sort_values('rmse').iloc[0]  # captura melhor linha da PARTE C
rmse_rs_c = best_c_exec['rmse'] * 1_000_000  # converte RMSE de R$ milhões para R$
print(f"Melhor PARTE C: {best_c_exec['modelo']}")  # imprime modelo vencedor da PARTE C
print(f"RMSE no TESTE: {best_c_exec['rmse']:.3f} R$ mi (≈ R$ {rmse_rs_c:,.0f})")  # imprime RMSE em duas escalas
print(f"MAE no TESTE : {best_c_exec['mae']:.3f} R$ mi")  # imprime MAE do vencedor


### Comparação final de modelos (PARTE C)

No gráfico final, cada linha é um modelo na janela de teste; use a legenda para ligar/desligar séries.

In [ ]:
plot_c = pd.DataFrame({'ds': test_ts['ds'], 'Real': y_test_ts, 'Baseline_Naive': pred_naive_ts, 'ETS_HoltWinters': pred_ets, 'ARIMAX_selic': pred_arimax})  # consolida séries para comparação
if pred_prophet is not None:  # inclui Prophet apenas quando houver previsão disponível
    plot_c['Prophet'] = pred_prophet

plot_c_long = plot_c.melt(id_vars='ds', var_name='serie_nome', value_name='valor')  # transforma para formato longo
fig_c_models = px.line(  # cria gráfico interativo multi-linhas da janela de teste
    plot_c_long,
    x='ds',
    y='valor',
    color='serie_nome',
    title='PARTE C — Previsão: Real vs Modelos (janela de teste)',
    labels={'ds': 'Data', 'valor': 'y (R$ milhões)', 'serie_nome': 'Série'},
    template='plotly_white'
)

rmse_line = ' | '.join([f"{r.modelo}:{r.rmse:.2f}" for r in results_c.itertuples() if pd.notna(r.rmse)])  # monta resumo de RMSE por modelo
fig_c_models.add_annotation(x=0.01, y=0.99, xref='paper', yref='paper', showarrow=False, text=f'RMSE -> {rmse_line}')  # adiciona resumo executivo no gráfico
fig_c_models.show()  # renderiza comparação final interativa

## 5) Conclusão didática

- **X -> y (regressão):** útil quando temos drivers para explicar a originação.
- **t -> y (série temporal):** útil quando o histórico da própria série é forte.
- **Regra prática:** validar no tempo e sempre comparar com baseline.


## 6) Checklist FP&A

- Baseline: compare sempre com referência simples.
- Split temporal: nunca embaralhar passado e futuro.
- Métricas: acompanhe MAE e RMSE no teste.
- Governança: documente dados, período e versão do modelo.
- Monitoramento: revise performance e recalibre quando necessário.


## 7) Resumo executivo (1 slide)

O que aprendemos hoje:
- escolher modelo é comparar **no TESTE temporal**;
- baseline é obrigatório para decisão robusta;
- o ranking muda quando o padrão do dado muda (linear vs não-linear);
- métricas + gráficos de erro + leitura de negócio formam a decisão FP&A.

O que levar para o trabalho amanhã:
- sempre começar com baseline + 2 modelos candidatos;
- validar em janela temporal recente;
- monitorar erro mensal e sinais de mudança de regime.


## Apêndice (Socorro): recuperar variáveis da PARTE B

Use este bloco **somente** se você pulou células e apareceu `NameError` na PARTE B.


In [ ]:
# Garantia para evitar NameError se alguma célula de treino tiver sido pulada
if 'pred_lr_b' not in globals():  # verifica se previsões da PARTE B existem
    model_lr_b = LinearRegression()  # recria LinearRegression da PARTE B
    model_lr_b.fit(X_train_nlin, y_train_nlin)  # retreina LinearRegression da PARTE B
    pred_lr_b = model_lr_b.predict(X_test_nlin)  # regenera previsão da LinearRegression da PARTE B
if 'pred_ridge_b' not in globals():  # verifica previsão do Ridge na PARTE B
    model_ridge_b = Ridge(alpha=1.0, random_state=42)  # recria Ridge da PARTE B
    model_ridge_b.fit(X_train_nlin, y_train_nlin)  # retreina Ridge da PARTE B
    pred_ridge_b = model_ridge_b.predict(X_test_nlin)  # regenera previsão do Ridge da PARTE B
if 'pred_en_b' not in globals():  # verifica previsão do ElasticNet na PARTE B
    model_en_b = ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42, max_iter=5000)  # recria ElasticNet da PARTE B
    model_en_b.fit(X_train_nlin, y_train_nlin)  # retreina ElasticNet da PARTE B
    pred_en_b = model_en_b.predict(X_test_nlin)  # regenera previsão do ElasticNet da PARTE B
if 'pred_tree_b' not in globals():  # verifica previsão da árvore na PARTE B
    model_tree_b = DecisionTreeRegressor(max_depth=4, random_state=42)  # recria árvore da PARTE B
    model_tree_b.fit(X_train_nlin, y_train_nlin)  # retreina árvore da PARTE B
    pred_tree_b = model_tree_b.predict(X_test_nlin)  # regenera previsão da árvore da PARTE B

### Continuação da recuperação (PARTE B)

Agora completamos a recuperação das previsões de ensemble para fechar o ranking da PARTE B.

In [ ]:
if 'pred_rf_b' not in globals():  # verifica previsão do RandomForest na PARTE B
    model_rf_b = RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42)  # recria RandomForest da PARTE B
    model_rf_b.fit(X_train_nlin, y_train_nlin)  # retreina RandomForest da PARTE B
    pred_rf_b = model_rf_b.predict(X_test_nlin)  # regenera previsão do RandomForest da PARTE B
if 'pred_gbm_b' not in globals():  # verifica previsão do GradientBoosting na PARTE B
    model_gbm_b = GradientBoostingRegressor(random_state=42)  # recria GradientBoosting da PARTE B
    model_gbm_b.fit(X_train_nlin, y_train_nlin)  # retreina GradientBoosting da PARTE B
    pred_gbm_b = model_gbm_b.predict(X_test_nlin)  # regenera previsão do GradientBoosting da PARTE B

### Recalcular tabela de métricas da PARTE B

Com todas as previsões disponíveis, recalculamos `results_b` para restaurar o ranking.

In [ ]:
results_b = pd.DataFrame([  # cria tabela de resultados da PARTE B
    {'modelo': 'Baseline_Naive', 'mae': mae_baseline_nlin, 'rmse': rmse_baseline_nlin},
    {'modelo': 'LinearRegression', 'mae': mean_absolute_error(y_test_nlin, pred_lr_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_lr_b))},
    {'modelo': 'Ridge', 'mae': mean_absolute_error(y_test_nlin, pred_ridge_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_ridge_b))},
    {'modelo': 'ElasticNet', 'mae': mean_absolute_error(y_test_nlin, pred_en_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_en_b))},
    {'modelo': 'DecisionTree', 'mae': mean_absolute_error(y_test_nlin, pred_tree_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_tree_b))},
    {'modelo': 'RandomForest', 'mae': mean_absolute_error(y_test_nlin, pred_rf_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_rf_b))},
    {'modelo': 'GradientBoosting', 'mae': mean_absolute_error(y_test_nlin, pred_gbm_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_gbm_b))},
]).sort_values('rmse').reset_index(drop=True)  # ordena por melhor RMSE
results_b  # exibe ranking final da PARTE B